In [ ]:
import random
import time

SIMILAR_MAP = {
    'A': ['A', '4', 'Λ', '∆'],
    'B': ['B', 'ß', 'β', '8'],
    'C': ['C', '(', 'ʗ', 'ϲ'],
    'D': ['D', 'Đ', 'ↁ'],
    'E': ['E', '3', '€', 'ε'],
    'F': ['F', 'Ƒ'],
    'G': ['G', '6', 'ɢ'],
    'H': ['H', 'ʰ', 'Ħ'],
    'I': ['I', '1', 'ɪ', '!'],
    'J': ['J', 'ʝ'],
    'K': ['K', 'Ⱪ', 'ʞ'],
    'L': ['L', '1', 'Ⱡ', 'Ł'],
    'M': ['M', 'ʍ', 'ϻ'],
    'N': ['N', 'ɴ', 'η'],
    'O': ['O', '0', 'Ø', 'Φ'],
    'P': ['P', 'Ƥ', 'ρ'],
    'Q': ['Q', 'ϙ', 'φ'],
    'R': ['R', 'ʀ', 'Я'],
    'S': ['S', '5', '$', '§'],
    'T': ['T', '7', 'ʇ'],
    'U': ['U', 'µ', 'υ'],
    'V': ['V', 'ʋ', '∨'],
    'W': ['W', 'ɯ', 'Ш'],
    'X': ['X', '×', '✕'],
    'Y': ['Y', 'Ɏ', '¥'],
    'Z': ['Z', '2', 'Ƶ'],
}

def get_variants(char):
    if char.upper() in SIMILAR_MAP:
        variants = SIMILAR_MAP[char.upper()]
        if char.islower():
            variants = [v.lower() for v in variants]
        variants[0] = char
        return variants
    else:
        return [char]

def initialize_population(original_text, pop_size):
    population = []
    variant_sets = [get_variants(c) for c in original_text]

    for _ in range(pop_size):
        chromosome = []
        for variants in variant_sets:
            rand_index = random.randint(0, len(variants) - 1)
            chromosome.append(rand_index)
        population.append(chromosome)
    return population

def compute_fitness(chromosome, original_text):
    replaced_count = sum(1 for gene in chromosome if gene != 0)
    return replaced_count

def roulette_wheel_selection(population, fitnesses):
    total_fitness = sum(fitnesses)
    if total_fitness == 0:
        return random.choice(population)

    pick = random.uniform(0, total_fitness)
    running_sum = 0.0
    for ind, fit in zip(population, fitnesses):
        running_sum += fit
        if running_sum >= pick:
            return ind
    return population[-1]

def single_point_crossover(parent1, parent2):
    if len(parent1) <= 1:
        return parent1[:], parent2[:]

    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(chromosome, original_text, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            variants = get_variants(original_text[i])
            chromosome[i] = random.randint(0, len(variants) - 1)
    return chromosome


def obfuscate_text_ga(original_text,
                      pop_size=10,
                      generations=10,
                      crossover_rate=0.7,
                      mutation_rate=0.05):
    # Initialize population
    population = initialize_population(original_text, pop_size)

    best_individual = None
    best_fitness = float('-inf')

    for gen in range(generations):
        # Evaluate fitness
        fitnesses = [compute_fitness(ind, original_text) for ind in population]

        # Track best
        for ind, fit in zip(population, fitnesses):
            if fit > best_fitness:
                best_fitness = fit
                best_individual = ind[:]

        print(f"Generation {gen} | Best fitness so far: {best_fitness}")

        # Create new population
        new_population = []
        while len(new_population) < pop_size:

            parent1 = roulette_wheel_selection(population, fitnesses)
            parent2 = roulette_wheel_selection(population, fitnesses)

            if random.random() < crossover_rate:
                child1, child2 = single_point_crossover(parent1, parent2)
            else:
                child1, child2 = parent1[:], parent2[:]

            child1 = mutate(child1, original_text, mutation_rate)
            child2 = mutate(child2, original_text, mutation_rate)

            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)

        population = new_population

    fitnesses = [compute_fitness(ind, original_text) for ind in population]
    for ind, fit in zip(population, fitnesses):
        if fit > best_fitness:
            best_fitness = fit
            best_individual = ind[:]

    return decode_chromosome(best_individual, original_text)


def decode_chromosome(chromosome, original_text):

    result_chars = []
    for i, gene in enumerate(chromosome):
        variants = get_variants(original_text[i])
        result_chars.append(variants[gene])
    return "".join(result_chars)

In [ ]:
random.seed(time.time())

text_to_obfuscate = "HELLO GA"

POP_SIZE = 10
GENERATIONS = 10
CROSSOVER_RATE = 0.7
MUTATION_RATE = 0.1

best_obfuscation = obfuscate_text_ga(text_to_obfuscate,
                                         pop_size=POP_SIZE,
                                         generations=GENERATIONS,
                                         crossover_rate=CROSSOVER_RATE,
                                         mutation_rate=MUTATION_RATE)

print("\nOriginal text:   ", text_to_obfuscate)
print("Best obfuscation:", best_obfuscation)

Generation 0 | Best fitness so far: 7
Generation 1 | Best fitness so far: 7
Generation 2 | Best fitness so far: 7
Generation 3 | Best fitness so far: 7
Generation 4 | Best fitness so far: 7
Generation 5 | Best fitness so far: 7
Generation 6 | Best fitness so far: 7
Generation 7 | Best fitness so far: 7
Generation 8 | Best fitness so far: 7
Generation 9 | Best fitness so far: 7

Original text:    HELLO GA
Best obfuscation: ʰεⱠⱠΦ ɢ∆


In [32]:
print(best_obfuscation)

ʰεⱠⱠΦ ɢ∆
